In [ ]:
# 
# Script to Download NSE Listed Companies Data and Store in SQLite Database from yfinance
#

In [5]:
# 1. Python Library Dependencies
# !pip install yfinance sqlalchemy pandas --quiet

import sys
import pandas as pd
import yfinance as yf
import sqlalchemy
import time
import os
import numpy as np


# --- CONFIGURATION ---
os.makedirs("data", exist_ok=True)                  # Ensuring existence of data directory
EXCEL_FILE = 'data/NSE_listed_companies_Filtered.xlsx'       # csv with names of companies
DB_NAME = 'data/NSE_Companies_filtered.db'

# 1. Configuration Parameters
START_DATE = '2004-09-01'            # start date
VOLATILITY_WINDOW = 30               # 30-Day Rolling Volatility

# 2. Connect to New Database
engine = sqlalchemy.create_engine(f'sqlite:///{DB_NAME}')
print(f"Created new database: {DB_NAME}")

# ==========================================
# PART 1: RESTORING SECTOR INFO
# ==========================================
print("\nPART 1: Creating 'stock_info' table from CSV...")

if not os.path.exists(EXCEL_FILE):
    print(f"CRITICAL: '{EXCEL_FILE}' not found.")
    sys.exit(1)

try:
    df = pd.read_excel(EXCEL_FILE)

    # Selecting relevant columns
    # Adjusting column names
    master_df = df[['TickerSymbol', 'Name', 'Sector']].copy()

    # Standardizing Column Names for SQL_smallcaps and using _
    master_df.columns = ['symbol', 'company_name', 'industry']

    # Adding .NS extension to symbol for yfinance
    master_df['symbol'] = master_df['symbol'].apply(lambda x: f"{x}.NS")

    # Saving to DB
    master_df.to_sql('stock_info', engine, if_exists='replace', index=False)

    # Retrieving list of tickers
    tickers = master_df['symbol'].tolist()
    print(f"Restored {len(tickers)} stocks into 'stock_info'.")

except Exception as e:
    print(f"Error reading CSV: {e}")
    sys.exit(1)

# ==========================================
# PART 2: DOWNLOADING 20 YEARS HISTORY
# ==========================================
print(f"\nPART 2: Downloading Daily Data (Start: {START_DATE})...")

success_count = 0
batch_size = 100                       # Downloading in Batches

# Looping through tickers
for i, ticker in enumerate(tickers):
    try:
        # A. Downloading Data
                                                                                # auto_adjust=True to handle Splits/Dividends
        df = yf.download(ticker, start=START_DATE, progress=False, interval = "1d",
                         auto_adjust=True, threads=True)

        if not df.empty:
            # Flatten columns
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [col[0] for col in df.columns]

            # B. Cleaning the Data
            df.reset_index(inplace=True)
            df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high',
                               'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)

            # Ensuring lowercasing of columns
            df.columns = [c.lower() for c in df.columns]

            # C. Feature Engineering (Volatility & Returns)
            # 1. Log Returns
            df['log_return'] = np.log(df['close'] / df['close'].shift(1))

            # 2. 5-30-60_Day Volatility
            df['volatility_5d'] = df['log_return'].rolling(window=5).std()                      # 5 day volatility
            df['volatility_30d'] = df['log_return'].rolling(window=VOLATILITY_WINDOW).std()     # 30 day volatility
            df['volatility_60d'] = df['log_return'].rolling(window=2*VOLATILITY_WINDOW).std()   # 60 day volatility

            # Dropping NaN rows generated by rolling window
            df.dropna(inplace=True)

            # D. Saving to Database
            # Table name = Ticker                                               (will save like "RELIANCE.NS"... so need to handle ticker names wtih EIKON separately)
            df.to_sql(ticker, engine, if_exists='replace', index=False)

            success_count += 1

        # Progress Update
        if i % 100 == 0:
            print(f"Processed {i}/{len(tickers)}: {ticker}")

    except Exception as e:
        print(f"Error {ticker}: {e}")

print(f"\n Download Complete ! Database created with {success_count} stocks.")

Created new database: data/NSE_Companies_filtered.db

PART 1: Creating 'stock_info' table from CSV...
Restored 2159 stocks into 'stock_info'.

PART 2: Downloading Daily Data (Start: 2004-09-01)...
Processed 0/2159: BEL.NS
Processed 100/2159: SPLPETRO.NS
Processed 200/2159: JYOTISTRUC.NS
Processed 300/2159: KSB.NS
Processed 400/2159: KMSUGAR.NS
Processed 500/2159: WSTCSTPAPR.NS
Processed 600/2159: SALSTEEL.NS



1 Failed download:
['TULSYAN.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-09-01 -> 2025-11-28)')


Processed 700/2159: KSERASERA.NS
Processed 800/2159: IPCALAB.NS


c:\Users\Akash Mittal\Documents\GitHub\AI Trade Challenge\my-ollama-project\.venv311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processed 900/2159: BSOFT.NS


...

Processed 2000/2159: 530617.NS



1 Failed download:
['530125.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['524703.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531637.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531173.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531015.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['519574.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['530665.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['524632.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['500672.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['514197.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['519307.NS']: YFTzMissingError('possibly delisted; no timez

Processed 2100/2159: 511644.NS



1 Failed download:
['531769.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['511447.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['517415.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531533.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['526443.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531126.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531960.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['532340.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['519463.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['531917.NS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['500346.NS']: YFTzMissingError('possibly delisted; no timez


 Download Complete ! Database created with 982 stocks.


In [5]:
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\Akash Mittal\Documents\GitHub\Quantamental Study\code
